In [ ]:

# Week 1: AI-Based Garbage Classification - Updated Code

# Step 1: Install required libraries
!pip install -q kaggle
!pip install -q tensorflow
!pip install -q matplotlib

# Step 2: Upload kaggle.json for API access
from google.colab import files
files.upload()  # Upload kaggle.json manually

# Step 3: Set up Kaggle API and download dataset
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!chmod 600 /content/kaggle.json
!kaggle datasets download -d farzadnekouei/trash-type-image-dataset
!unzip -oq trash-type-image-dataset.zip -d data

# Step 4: Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Step 5: Prepare dataset paths (corrected path)
train_path = "data/TrashType_Image_Dataset"
val_path = "data/TrashType_Image_Dataset"  # Using same path as there's no split

# Step 6: Create ImageDataGenerator instances
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Step 7: Build CNN Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  # 6 classes
])

# Step 8: Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 9: Train Model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

# Step 10: Plot accuracy and loss
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()
